**Impfdaten hinzufügen und Datumsformat ändern zu (dd-mm-yyyy)**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 250)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install xlrd

In [ ]:
data = pd.read_excel('/content/drive/My Drive/Manu Master/Datasetforlabelling/vaccinationsaccordingtocountry.xlsx')

data['daily_vaccinations'] = data['daily_vaccinations'].fillna(0)
data['daily_vaccinations_per_million'] = data['daily_vaccinations_per_million'].fillna(0)

data[["daily_vaccinations", "daily_vaccinations_per_million"]] = data[["daily_vaccinations", "daily_vaccinations_per_million"]].apply(pd.to_numeric)

data["daily_vaccinations"] = data["daily_vaccinations"].astype(np.int64)
data["daily_vaccinations_per_million"] = data["daily_vaccinations_per_million"].astype(np.int64)

#df['column name'] = df['column name'].astype(np.int64)
countrylist = data[["country"]].values.ravel()
countrylist =  pd.unique(countrylist)

data['Variation'] = (data.sort_values('date').groupby(['country'])['daily_vaccinations']
                   .transform(lambda x: x.diff()))


#data['A_dif'] = data['daily_vaccinations'].diff()
#data['A_dif'] = data['A_dif'].fillna(0)
#data['A_dif'] = data['A_dif'].astype(np.int64)
mask = (data['date'] > '2020-12-12') & (data['date'] <= '2021-08-28')
data = (data.loc[mask])

data

,country,iso_code,date,daily_vaccinations,daily_vaccinations_per_million,Variation
0,Afghanistan,AFG,2021-02-22,0,0,NaN
1,Afghanistan,AFG,2021-02-23,13670,340,13670.0
2,Afghanistan,AFG,2021-02-24,13670,340,0.0
3,Afghanistan,AFG,2021-02-25,13670,340,0.0
4,Afghanistan,AFG,2021-02-26,13670,340,0.0
...,...,...,...,...,...,...
43871,Zimbabwe,ZWE,2021-08-24,626380,41500,-63660.0
43872,Zimbabwe,ZWE,2021-08-25,675970,44790,49590.0
43873,Zimbabwe,ZWE,2021-08-26,606880,40210,-69090.0
43874,Zimbabwe,ZWE,2021-08-27,537840,35640,-69040.0


In [ ]:
flrslt_df = '/content/drive/My Drive/Manu Master/Datasetforlabelling/vaccinatedwithdifferencew.csv'
data.to_csv(flrslt_df,index=False) 

In [ ]:
tweettoclean = pd.read_csv("/content/drive/My Drive/Manu Master/Datasetforlabelling/datacompleted.csv")

In [ ]:
tweettoclean= tweettoclean.sort_values(by="date")
df1 = tweettoclean[['country', 'date','people_fully_vaccinated', 'daily_vaccinations']]
df1

In [ ]:
flrslt_df = '/content/drive/My Drive/Manu Master/Datasetforlabelling/countryandvaccinated.csv'
df1.to_csv(flrslt_df,index=False) 

In [ ]:
tweettoclean.drop(['user_name','user_location','user_description','user_created','user_verified','hashtags','is_retweet','gcode','lat','long'], axis=1, inplace=True)


In [ ]:
tweettoclean['date'] = pd.to_datetime(tweettoclean['date'] ,errors = 'coerce',format = '%Y-%m-%d').dt.strftime("%Y-%m-%d")

In [ ]:
flrslt_df = '/content/drive/My Drive/Manu Master/Datasetforlabelling/datacompletedwithdate.csv'
tweettoclean.to_csv(flrslt_df,index=False) 

In [ ]:
tweettoclean = tweettoclean.replace(r'\n',' ', regex=True) 

In [ ]:
nltk.download("stopwords") 
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
pat = r'\b(?:{})\b'.format('|'.join(stop))
tweettoclean['text'] = tweettoclean['text'].str.replace(pat, '')
tweettoclean['text'] = tweettoclean['text'].str.replace(r'\s+', ' ')
#tweet_df = tweet_df[tweet_df['text'].str.replace(r'\s+', ' ')]
tweettoclean['text']= (tweettoclean['text']).astype(str).str.replace('.', '')
misspell_data = pd.read_csv("/content/drive/My Drive/Manu Master/lstm/aspell.txt",sep=":",names=["correction","misspell"])
misspell_data.misspell = misspell_data.misspell.str.strip()
misspell_data.misspell = misspell_data.misspell.str.split(" ")
misspell_data = misspell_data.explode("misspell").reset_index(drop=True)
misspell_data.drop_duplicates("misspell",inplace=True)
miss_corr = dict(zip(misspell_data.misspell, misspell_data.correction))

#Sample of the dict
#{v:miss_corr[v] for v in [list(miss_corr.keys())[k] for k in range(20)]}
def misspelled_correction(val):
    for x in val.split(): 
        if x in miss_corr.keys(): 
            val = val.replace(x, miss_corr[x]) 
    return val

tweettoclean["text"] = tweettoclean.text.apply(lambda x : misspelled_correction(x))

In [ ]:
!pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.MENTION, p.OPT.URL)
!pip install emoji
import emoji

In [ ]:
def punctuation(val): 
  
    punctuations = '''()-[]{};:'"\,<>./@#$%^&_~'''
  
    for x in val.lower(): 
        if x in punctuations: 
            val = val.replace(x, " ") 
    return val

tweettoclean["text"] =tweettoclean.text.apply(lambda x : p.clean(x))
tweettoclean["text"]  = tweettoclean.text.apply(lambda x : ' '.join(punctuation(emoji.demojize(x)).split()))

In [ ]:
tweettoclean

,date,text,bert_emotion,country,valence,kulturerde
0,2020-12-12,FDA authorizes PfizerBioNTech coronavirus vaccine emergency use,anticipation,Egypt,Positive,Orient
1,2020-12-12,Trump announces vaccine rollout less 24 hours The first Americans vaccinated …,anticipation,United Kingdom,Positive,Europe
2,2020-12-12,one side effects PfizerBioNTech …,trust,United Kingdom,Positive,Europe
3,2020-12-12,An immigrant Muslim couple find COVID 19 Vaccine CovidVaccine PfizerBioNTech Pfizervaccine PfizerProud,anticipation,United Kingdom,Positive,Europe
4,2020-12-12,Vaccine!! Anyone?? covid Pfizervaccine PfizerBioNTech,surprise,United Kingdom,Positive,Europe
...,...,...,...,...,...,...
135505,2021-08-29,EMBARRASSING Ontario reporting 740 case COVID19 141 new case Toronto Canada politicizing vaccine Ford Trudeau order Russia SputnikV end business lockdown drug addiction violence USA Paralympics MentalHealthMatters sundayvibes cdnpoli,surprise,Canada,Positive,Angloamerika
135506,2021-08-29,When COVAXIN get WHO approval? It impact many people whose job requires abroad travel Please update common people WHO approval status ASAP!!!,fear,India,Negative,South Asia
135507,2021-08-29,MegaVaccinationDrive COVAXIN organised shivsena yuvasena,anticipation,India,Positive,South Asia
135508,2021-08-29,My oldest knows guy works Pfizer says vaccine safe since would anyone works BigPharma tell us TRUTH? resist NewWorldOrder GreatReset JustSayNO tyranny PoliceState SocialConditioning Fascism VaccineApartheid segregation,disgust,Canada,Negative,Angloamerika


In [ ]:
flclean = '/content/drive/My Drive/Manu Master/dataset/cleaneddata.csv'
tweettoclean.to_csv(flclean,index=False) 

# Bitte Geolocation I anschauen